# Yatir land surface parameterization

In [ ]:
from timutils.git_tools import print_cwd_git_version
print_cwd_git_version()

In [ ]:
import holoviews as hv
from holoviews import opts
import os
from cartopy import crs as ccrs

from geoviews_tools import yatir_landuse_to_xarray

In [ ]:
hv.extension('bokeh')

#allow multiple plots in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
landuse = yatir_landuse_to_xarray()

In [ ]:
import geoviews as gv
import geoviews.feature as gf
lat_dim = hv.Dimension('south_north', label='Latitude', unit='deg N')
lon_dim = hv.Dimension('west_east', label='Longitude', unit='deg E')

In [ ]:
# TODO
#  X use Quadmesh, not Image
#  - index by lat, lon (not array index)
#  - show surrounding map
#  - draw box for Yatir on d02 maps
#  - better colormap
def make_maps(landuse, domain, scale='50m'):
    """make a list of maps, one per WRF run, for the specified domain"""
    wrf_runs = landuse[domain].coords['WRFrun'].values
    maps_list = [gv.Dataset(landuse[domain].sel(WRFrun=this_run), kdims=['PFT', 'lon', 'lat']).to(
        gv.QuadMesh, 
        ['lon', 'lat'], 
        'LANDUSEF', 
        label=this_run + ", " + domain).opts(colorbar=True, cmap='Blues') #, projection=ccrs.PlateCarree()) 
                 for this_run in wrf_runs
                ]
    # draw a box around domain 03
    corner_indices = ([0, 0, -1, -1], [0, -1, -1, -0])
    d03_corners = list(zip(landuse['d03']['lon'].values[corner_indices], 
                           landuse['d03']['lat'].values[corner_indices]))
    d03_box = gv.Polygons(d03_corners).opts(style={'line_color': '#d95f02', 'color': None}) 
#     for this_run in range(len(wrf_runs)):
#         maps_list[this_run] = maps_list[this_run] * d03_box * gf.coastline(scale=scale) * gf.borders(scale=scale)
#         maps_list[this_run].opts(padding=0.1)
    return(maps_list)
d02_maps = make_maps(landuse, 'd02')
d03_maps = make_maps(landuse, 'd03', scale='10m')

In [ ]:
# hv.Layout(d02_maps)

In [ ]:
# hv.Layout(d03_maps)

In [ ]:
def make_one_map(landuse, domain, WRF_run, scale='50m', varname='LANDUSEF', vardim=None):
    """make a list of maps, one per WRF run, for the specified domain"""
    if vardim is None:
        vardim=hv.Dimension(varname)
    this_map = gv.Dataset(landuse[domain].sel(WRFrun=WRF_run), 
                          kdims=['PFT', 'lon', 'lat']).to(
        gv.QuadMesh, 
        kdims=['lon', 'lat'], 
        vdims=vardim).opts(colorbar=True, cmap='Blues', projection=ccrs.PlateCarree())
                 
    # draw a box around domain 03
    corner_indices = ([0, 0, -1, -1], [0, -1, -1, -0])
    d03_corners = list(zip(landuse['d03']['lon'].values[corner_indices], 
                           landuse['d03']['lat'].values[corner_indices]))
    d03_box = gv.Polygons(d03_corners).opts(style={'line_color': '#d95f02', 'color': None}) 
    return(this_map * d03_box * gf.coastline(scale=scale) * gf.borders(scale=scale))
    #return(this_map)

In [ ]:
vardim = hv.Dimension('LANDUSEF', label='Land Use Fraction', range=(0.0, 1.0))

In [ ]:
landusef_dict = {(WRFdomain, WRFrun): make_one_map(landuse, WRFdomain, WRFrun, scale='110m', vardim=vardim) 
                for WRFdomain in ['d02', 'd03'] 
                for WRFrun in ['ctl', 'ytr']}

In [ ]:
kdims = [hv.Dimension(('WRFdomain', 'WRF domain'), default='d03'),
         hv.Dimension(('WRFrun', 'WRF run'), default='ctl')]
holomap = hv.HoloMap(landusef_dict, kdims=kdims)


### Land use fraction ###

0.0 to 1.0 for each land cover type

In [ ]:
gv.GridSpace(holomap).opts(opts.GridSpace(plot_size=200, title='Land Use Fraction'))

### Dominant land use type ###

In [ ]:
from map_tools_twh.map_tools_twh import get_IGBP_modMODIS_21Category_PFTs_cmap, get_IGBP_modMODIS_21Category_PFTs_table
from bokeh.models import FixedTicker
import numpy as np

cmap = get_IGBP_modMODIS_21Category_PFTs_cmap()
color_table = get_IGBP_modMODIS_21Category_PFTs_table()
vardim = hv.Dimension('LU_INDEX', 
                      label='Dominant Land Use',
                      values=[color_table['long_name'][x] for x in range(21)],
                      range=(0.0, 20.0), step=1.0)
lu_idx_dict = {(WRFdomain, WRFrun): make_one_map(landuse, WRFdomain, WRFrun, scale='110m', varname="LU_INDEX", vardim=vardim) 
                for WRFdomain in ['d02', 'd03'] 
                for WRFrun in ['ctl', 'ytr']}
holomap = hv.HoloMap(lu_idx_dict, kdims=kdims)

#gv.GridSpace(holomap).opts(opts.GridSpace(plot_size=200, title='Dominant Land Use'), opts.QuadMesh(cmap=cmap, colorbar=True))

In [ ]:
min_PFT = 0
max_PFT = 21
tick_locations = np.linspace(0.5, 20.5, 21)
colorbar_opts={'major_label_overrides': {tick_locations[i]: color_table['long_name'][i]
                                         for i in range(max_PFT)},
              'ticker': FixedTicker(ticks=tick_locations),
              'major_label_text_align': 'left'}
map_d03_ytr = lu_idx_dict['d03', 'ytr']
map_d03_ytr.opts(opts.QuadMesh(cmap=cmap, 
                               colorbar=True, 
                               width=500, 
                               height=300, 
                               colorbar_opts=colorbar_opts)).redim.range(LU_INDEX=(min_PFT, max_PFT))